<a href="https://colab.research.google.com/github/Cnaraa/websiteClassification/blob/main/translated_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")

model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Используем устройство: cuda


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate_long_text(texts, max_length=256, batch_size=128):
    translated_texts = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]


        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)


        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=4,
                early_stopping=True,
                eos_token_id=model.config.eos_token_id
            )

        translated_batch = [
            tokenizer.decode(ids, skip_special_tokens=True).strip() for ids in outputs
        ]
        translated_texts.extend(translated_batch)

    return translated_texts

In [ ]:
df = pd.read_csv('/content/translated_part_10.csv')
df = df.head(50)


start_time = time.time()


df.loc[:, 'translated_summary'] = translate_long_text(df['summary'].tolist(), max_length=256)


end_time = time.time()
print(f"Общее время выполнения: {end_time - start_time:.2f} секунд")


print(df[['summary', 'translated_summary']])

Общее время выполнения: 11.69 секунд
                                              summary  \
0   Constantly locating moving objects, i.e., geos...   
1   The mimicking of human-like arm movement chara...   
2   Humans' ability to smoothly switch between loc...   
3   In this paper, we introduce RealDex, a pioneer...   
4   Bird's-eye view (BEV) maps are an important ge...   
5   This paper describes the design of a miniature...   
6   Motion trajectories offer reliable references ...   
7   Perceiving and understanding highly dynamic an...   
8   Relative drone-to-drone localization is a fund...   
9   Unmanned ground vehicle obstacle crossing gene...   
10  The ability of a soft robot to perform specifi...   
11  This work focuses on the dual-arm object rearr...   
12  In recent years, object-oriented simultaneous ...   
13  Autonomous vehicles trained through Multi-Agen...   
14  Learning highly dynamic behaviors for robots h...   
15  Interactive imitation learning is an efficient.

In [ ]:
base_path = "/content/"
output_path = "/content/"

In [ ]:
torch.cuda.empty_cache()
from google.colab import files

In [ ]:
for part in range(10, 11):
    file_name = f"translated_part_{part}.csv"
    output_file = f"translated_data_part_{part}.csv"

    print(f"Обработка файла: {file_name}")

    try:

        df = pd.read_csv(base_path + file_name)


        start_time = time.time()
        df.loc[:, 'translated_summary'] = translate_long_text(df['summary'].tolist(), max_length=256, batch_size=64)
        end_time = time.time()


        df.to_csv(output_path + output_file, index=False)
        print(f"Файл {file_name} успешно переведен и сохранен как {output_file}")
        print(f"Время выполнения: {end_time - start_time:.2f} секунд\n")
        files.download(f"/content/translated_data_part_10.csv")

    except Exception as e:
        print(f"Ошибка при обработке файла {file_name}: {e}\n")
        continue


Обработка файла: translated_part_10.csv
Файл translated_part_10.csv успешно переведен и сохранен как translated_data_part_10.csv
Время выполнения: 4882.37 секунд



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>